<a href="https://colab.research.google.com/github/rajeshvelicheti/ml/blob/main/Building_an_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# House-keeping

In [2]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00


# Pre-Processing

In [3]:
import urllib.request

# Fetch the-verdict.txt

url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x7dcdcaaf5c30>)

In [4]:
with open(file_path, "r") as f:
    raw_text = f.read()

print('Total characters', len(raw_text))
print(raw_text[:99])

Total characters 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
#Pre=processing text and splitting to words
import re
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4649


In [6]:
preprocessed[:20]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow',
 'enough',
 '--',
 'so',
 'it',
 'was']

In [7]:
# Find unique words for token id generation
all_words = sorted(set(preprocessed))
all_words.extend(["<|endoftext|>", "<|unk|>"])
vocab_size = len(all_words)
print(vocab_size)

1161


In [8]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [9]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [10]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted cyan, you know,"
       Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[1, 58, 2, 872, 1013, 615, 541, 763, 1160, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 773, 812, 7]
" It' s the last he painted <|unk|>, you know," Mrs. Gisburn said with pardonable pride.


## Byte-Pair encoding

In [11]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

def BPEEncode(input):
  return tokenizer.encode(input, allowed_special={"<|endoftext|>"})

def BPEDecode(input):
  return tokenizer.decode(input)


In [13]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
text= "Akwirw ier."
integers = BPEEncode(text)
print(integers)
strings = BPEDecode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959, 13]
Akwirw ier.


In [15]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))
enc_sample = enc_text[50:]

5145


In [18]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")
print(BPEDecode(x))
print(BPEDecode(y))

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]
 and established himself in
 established himself in a


In [19]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
#GPTDatasetV1(raw_text, tokenizer, max_length=128, stride=1)

some_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in range(0, 10 - 4, 1):
  print("Input")
  print(some_list[i:i+4])
  print('Target')
  print(some_list[i+1:i+5])

In [34]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=0
    )

    return dataloader

In [45]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [49]:
import torch

vocab_size = 6
output_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)
print(embedding_layer(torch.tensor([3])))

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [50]:
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [51]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
   stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [69]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)
token_embeddings

torch.Size([8, 4, 256])


tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.3163,  ..., -1.1582, -1.13

In [70]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)
pos_embeddings

torch.Size([4, 256])


tensor([[ 0.6415,  1.2947, -0.5925,  ...,  1.0051, -0.8827,  1.5732],
        [ 1.1415,  1.5395,  0.3079,  ..., -2.1034, -0.8637,  0.4833],
        [ 1.1637,  1.3017,  0.1629,  ...,  0.6582,  0.0820,  0.5359],
        [-1.0371,  1.6309, -0.4921,  ..., -0.5899,  0.5671,  0.7218]],
       grad_fn=<EmbeddingBackward0>)

In [75]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)
input_embeddings

torch.Size([8, 4, 256])


tensor([[[ 1.1328,  2.4186,  0.8663,  ...,  0.6055, -2.7563,  1.4286],
         [ 1.5896,  1.7931,  0.0424,  ..., -1.6037, -2.0628, -0.7011],
         [ 0.9130,  1.2470,  0.8317,  ...,  1.6200,  2.4557,  0.4831],
         [-0.0914,  2.4966,  1.1270,  ..., -1.0443, -0.1789,  1.0701]],

        [[ 2.1875,  3.0316, -1.3773,  ...,  0.9046, -0.0243,  1.2311],
         [-0.7207,  1.3481, -0.0733,  ..., -0.9814, -1.2133,  1.0924],
         [ 3.1484,  0.6534,  0.0215,  ...,  0.2741, -0.8535,  1.9838],
         [-0.0724,  2.9282, -2.1129,  ...,  0.5564,  2.1468,  1.1187]],

        [[-0.1298,  1.9519, -0.4262,  ...,  0.2007, -0.8286,  2.3158],
         [ 1.9461,  2.0442,  1.6001,  ..., -0.6386, -0.4540,  0.8038],
         [ 1.2431, -0.4619,  0.7379,  ...,  2.8405,  1.9051,  0.1725],
         [-0.6104,  1.5662,  0.0764,  ..., -1.1108,  1.8736,  1.5691]],

        ...,

        [[-0.9741,  2.2557, -3.2362,  ...,  0.0406,  0.2061,  3.2115],
         [ 0.7430,  0.6160, -1.0084,  ..., -3.2616, -1.99

In [55]:
torch.arange(4)

tensor([0, 1, 2, 3])